# Creating and modelling metallic supercells

In this section we will be concerned with modelling supercells of aluminium.
When dealing with periodic problems there is no unique definition of the
lattice: Clearly any duplication of the lattice along an axis is also a valid
repetitive unit to describe exactly the same system.
This is exactly what a **supercell** is: An $n$-fold repetition along one of the
axes of the original lattice.

The following code achieves this for aluminium:

In [1]:
using DFTK
using LinearAlgebra

function aluminium_setup(repeat=1; Ecut=7.0, kgrid=[2, 2, 2])
    a = 7.65339
    lattice = a * Matrix(I, 3, 3)
    Al = ElementPsp(:Al, psp=load_psp("hgh/lda/al-q3"))
    atoms     = [Al, Al, Al, Al]
    positions = [[0.0, 0.0, 0.0], [0.0, 0.5, 0.5], [0.5, 0.0, 0.5], [0.5, 0.5, 0.0]]

    # Make supercell in ASE:
    # We convert our lattice to the conventions used in ASE
    # and then back ...
    supercell = ase_atoms(lattice, atoms, positions) * (repeat, 1, 1)
    lattice   = load_lattice(supercell)
    positions = load_positions(supercell)
    atoms = fill(Al, length(positions))

    # Construct an LDA model and discretise
    # Note: We disable symmetries explicitly here. Otherwise the problem sizes
    #       we are able to run on the CI are too simple to observe the numerical
    #       instabilities we want to trigger here.
    model = model_LDA(lattice, atoms, positions; temperature=1e-3, symmetries=false)
    PlaneWaveBasis(model; Ecut, kgrid)
end;

As part of the code we are using a routine inside the ASE,
the [atomistic simulation environment](https://wiki.fysik.dtu.dk/ase/index.html)
for creating the supercell and make use of the two-way interoperability of
DFTK and ASE. For more details on this aspect see the documentation
on Input and output formats.

Write an example supercell structure to a file to plot it:

In [2]:
setup = aluminium_setup(5)
ase_atoms(setup.model).write("al_supercell.png")

<img src="https://docs.dftk.org/stable/examples/al_supercell.png" width=500 height=500 />

As we will see in this notebook the modelling of a system generally becomes
harder if the system becomes larger.

- This sounds like a trivial statement as *per se* the cost per SCF step increases
  as the system (and thus $N$) gets larger.
- But there is more to it:
  If one is not careful also the *number of SCF iterations* increases
  as the system gets larger.
- The aim of a proper computational treatment of such supercells is therefore
  to ensure that the **number of SCF iterations remains constant** when the
  system size increases.

For achieving the latter DFTK by default employs the `LdosMixing`
preconditioner [^HL2021] during the SCF iterations. This mixing approach is
completely parameter free, but still automatically adapts to the treated
system in order to efficiently prevent charge sloshing. As a result,
modelling aluminium slabs indeed takes roughly the same number of SCF iterations
irrespective of the supercell size:

[^HL2021]:
   M. F. Herbst and A. Levitt.
   *Black-box inhomogeneous preconditioning for self-consistent field iterations in density functional theory.*
   J. Phys. Cond. Matt *33* 085503 (2021). [ArXiv:2009.01665](https://arxiv.org/abs/2009.01665)

In [3]:
is_converged = DFTK.ScfConvergenceDensity(1e-4)  # Flag convergence based on density
self_consistent_field(aluminium_setup(1); is_converged);

n     Energy            log10(ΔE)   log10(Δρ)   Diag
---   ---------------   ---------   ---------   ----
  1   -8.298382608046                   -0.85    5.1
  2   -8.300188408521       -2.74       -1.26    1.2
  3   -8.300446849016       -3.59       -1.89    3.4
  4   -8.300461831448       -4.82       -2.73    1.6
  5   -8.300464106799       -5.64       -3.05    4.9
  6   -8.300464350354       -6.61       -3.19    8.2
  7   -8.300464509911       -6.80       -3.35    1.0
  8   -8.300464567448       -7.24       -3.45    2.0
  9   -8.300464626769       -7.23       -3.67    1.1
 10   -8.300464631371       -8.34       -3.75    1.0
 11   -8.300464640867       -8.02       -3.98    1.0
 12   -8.300464643751       -8.54       -4.43    1.0


In [4]:
self_consistent_field(aluminium_setup(2); is_converged);

n     Energy            log10(ΔE)   log10(Δρ)   Diag
---   ---------------   ---------   ---------   ----
  1   -16.65994426391                   -0.70    6.1
  2   -16.66414654585       -2.38       -1.14    1.0
  3   -16.67921076143       -1.82       -1.91    2.8
  4   -16.67928290322       -4.14       -2.70    5.6
  5   -16.67928609509       -5.50       -3.01    5.2
  6   -16.67928619953       -6.98       -3.49    1.0
  7   -16.67928621488       -7.81       -3.90    1.8
  8   -16.67928622287       -8.10       -4.54    1.0


In [5]:
self_consistent_field(aluminium_setup(4); is_converged);

n     Energy            log10(ΔE)   log10(Δρ)   Diag
---   ---------------   ---------   ---------   ----
  1   -33.32764500165                   -0.56    7.1
  2   -33.33457244921       -2.16       -1.00    1.5
  3   -33.33605003969       -2.83       -1.72    4.4
  4   -33.33690316141       -3.07       -2.47    8.2
  5   -33.33694261105       -4.40       -3.13    6.9
  6   -33.33694389327       -5.89       -3.89    5.2
  7   -33.33694392576       -7.49       -4.43    5.5


When switching off explicitly the `LdosMixing`, by selecting `mixing=SimpleMixing()`,
the performance of number of required SCF steps starts to increase as we increase
the size of the modelled problem:

In [6]:
self_consistent_field(aluminium_setup(1); is_converged, mixing=SimpleMixing());

n     Energy            log10(ΔE)   log10(Δρ)   Diag
---   ---------------   ---------   ---------   ----
  1   -8.298440794154                   -0.85    4.9
  2   -8.300265042105       -2.74       -1.58    1.0
  3   -8.300423442130       -3.80       -2.40    4.9
  4   -8.300322867210   +   -4.00       -2.18   10.9
  5   -8.300464379215       -3.85       -3.58    2.6
  6   -8.300464591398       -6.67       -3.93    3.9
  7   -8.300464637297       -7.34       -4.38    2.1


In [7]:
self_consistent_field(aluminium_setup(4); is_converged, mixing=SimpleMixing());

n     Energy            log10(ΔE)   log10(Δρ)   Diag
---   ---------------   ---------   ---------   ----
  1   -33.32631060646                   -0.56    6.8
  2   -33.32960098729       -2.48       -1.27    1.2
  3   -26.06323343365   +    0.86       -0.64    8.6
  4   -33.03947848011        0.84       -1.26    5.6
  5   -33.29781079349       -0.59       -1.47    4.8
  6   -33.04331826011   +   -0.59       -1.33    4.0
  7   -33.28337696938       -0.62       -1.68    5.2
  8   -33.33509294978       -1.29       -2.34    3.0
  9   -33.33493952572   +   -3.81       -2.36    2.4
 10   -33.33551371535       -3.24       -2.40    1.8
 11   -33.33621821987       -3.15       -2.57    1.8
 12   -33.33686901404       -3.19       -3.00    3.0
 13   -33.33691771855       -4.31       -3.22    3.2
 14   -33.33694025200       -4.65       -3.45    3.0
 15   -33.33694008882   +   -6.79       -3.69    3.8
 16   -33.33694343902       -5.47       -4.13    2.6


For completion let us note that the more traditional `mixing=KerkerMixing()`
approach would also help in this particular setting to obtain a constant
number of SCF iterations for an increasing system size (try it!). In contrast
to `LdosMixing`, however, `KerkerMixing` is only suitable to model bulk metallic
system (like the case we are considering here). When modelling metallic surfaces
or mixtures of metals and insulators, `KerkerMixing` fails, while `LdosMixing`
still works well. See the Modelling a gallium arsenide surface example
or [^HL2021] for details. Due to the general applicability of `LdosMixing` this
method is the default mixing approach in DFTK.